In [9]:
import importlib
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
from scipy.special import expit

from mapra import prep

from multiprocessing import Process
from time import sleep

from sklearn import linear_model, metrics
from sklearn.metrics import auc, plot_roc_curve
from sklearn.model_selection import train_test_split, StratifiedKFold

sns.set_theme(style='white')

In [10]:
importlib.reload(prep)
data = prep.dataset()


/home/quirin/PYTHON/mapra


In [11]:
def do_work(extend, modify, metric, func, thresholds):

    df = data.dataframe_repeats_avg(reduced=True)

    mbeds = data.fetch_df_with_pairwise_distances(extend=extend, df=df, modify=modify, func=func)
    """
    :param extend: The number of additional neighbours to include on each side
    :param df: optional dataframe, otherwise will be dataframe_abbrev(reduced=reduced)
    :param reduced: default is True, if only the redundancy-reduced dataset shall be loaded from the df
    :param modify: 'flip' distances for negative changes, use the 'abs' value, only 'pos' or 'neg'
    :param scaler: 'std' or 'minmax'
    :param func: the function to handle compound mutations: np.mean, sum, max, min
    :return:
    """
    # mbeds.data


    # and convert changes to discrete classes
    # y_train = y_train.apply(lambda f: '--' if f < -10 else '-' if f < -1 else '=' if f < 1 else '+' if f < 10 else '++')
    # y_train = y_train.apply(lambda f: -1 if f <= -1 else 0 if f <= 1 else 1)
    # y_train = y_train.apply(lambda f: -1 if f <= 0 else 1)
    # y_train = y_train.apply(lambda f: 0 if abs(f) <= 2 else 1)

    # metric = 'euclidean'
    df = mbeds.data.loc[(mbeds.data.metric == mbeds.metric_labels.index(metric)),
                        [c for c in mbeds.data.columns if c != 'metric']]
    # # create class labels
    # thresholds = [3, 1, 1.5]
    df['label'] = df.apply(lambda gdf: 0 if abs(gdf.change) <= thresholds[int(gdf.delta)] else 1, axis=1)

    # plotting prep
    fig, axs = plt.subplots(3, 4, figsize=(14, 12), gridspec_kw={'width_ratios': [1, 1, 1, .4]})
    cmap = sns.color_palette('viridis', 3)
    mean_fpr = np.linspace(0, 1, 100)
    tprs, aucs = list(), list()
    test_size = .2

    for i, delta in enumerate(mbeds.delta_labels):
        # select the records for this delta
        dfn = df.loc[df.delta == i]

        sns.regplot(data=dfn,
                ax=axs[i, 0],
                x='dist', y='change',
                color=cmap[i],
                marker='+',
                scatter_kws={'s': 3, 'alpha': .2},
                fit_reg=False, logistic=True)
        axs[i, 0].set(ylabel=data.tex_lookup[delta], xlabel='')
        axs[i, 0].axhline(y=thresholds[i], lw=2, color='.5', alpha=.8)

        # make the scatterplot
        sns.regplot(data=dfn,
                    ax=axs[i, 1],
                    x='dist', y='label',
                    marker='+',
                    y_jitter=.06, color=cmap[i],
                    scatter_kws={'s': 3, 'alpha': .2},
                    fit_reg=False, logistic=True)
        axs[i, 1].set(xlabel='', yticks=[0, 1], ylabel='',
                      yticklabels=[f'≤ {thresholds[i]}', f'> {thresholds[i]}'])
        axs[i, 1].tick_params(axis='y', labelrotation=90)
        axs[i, 1].yaxis.labelpad = -10

        # fetch the training data
        X, y = np.array(dfn['dist']).reshape(-1, 1), np.array(dfn['label'])

        # split into test and training data
        X, x_test, y, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42)

        n_splits=8
        cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        clf = linear_model.LogisticRegressionCV(random_state=42,
                                                cv=cv,  # use existing splitter
                                                refit=True)  # build best model in the end

        for j, (train, test) in enumerate(cv.split(X, y)):
            clf.fit(X[train], y[train])

            # plot a ROC curve
            viz = plot_roc_curve(clf, X[test], y[test], lw=1, color='.5', alpha=.3, ax=axs[i, 2])
            interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            aucs.append(viz.roc_auc)

            # plot the loss curves
            x_line = np.linspace(0, max(X), 100)
            loss = expit(x_line * clf.coef_ + clf.intercept_).ravel()
            axs[i, 1].plot(x_line, loss, lw=1, color='.5', alpha=.3)

        # correlations
        pearson_corr = np.corrcoef(dfn[['dist', 'change']], rowvar=False)[0, 1]
        axs[i, 0].text(.07, .9, f'corr: {pearson_corr:.2f}', transform=axs[i, 0].transAxes)

        # label class sizes
        axs[i, 1].text(.07, .75, f'{len(dfn.loc[dfn.label == 1]):.0f}', transform=axs[i, 1].transAxes)
        axs[i, 1].text(.07, .20, f'{len(dfn.loc[dfn.label == 0]):.0f}', transform=axs[i, 1].transAxes)

        # plot the overall loss curve
        x_line = np.linspace(0, max(X), 100)
        loss = expit(x_line * clf.coef_ + clf.intercept_).ravel()
        axs[i, 1].plot(x_line, loss, lw=2.5, color=cmap[i], alpha=1)

        # plot the diagonal
        axs[i, 2].plot([0, 1], [0, 1], lw=1, color='.5', alpha=.6)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        axs[i, 2].plot(mean_fpr, mean_tpr, color=cmap[i], lw=2.5, alpha=1)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        axs[i, 2].fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2)

        axs[i, 2].set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05])  #, title="Receiver operating characteristic example")
        axs[i, 2].text(.04, .85, f'AUC= {mean_auc:0.2f}$\pm${std_auc:.2f}\n{n_splits}×CV', transform=axs[i, 2].transAxes)
        axs[i, 2].get_legend().remove()
        axs[i, 2].set(xlabel='FPR', ylabel='TPR', xticks=[0, 1], yticks=[0, 1])
        axs[i, 2].xaxis.labelpad = -10
        axs[i, 2].yaxis.labelpad = -10

        # make final test
        f = metrics.plot_confusion_matrix(clf, x_test, y_test, ax=axs[i, 3],
                                          cmap='binary', colorbar=False)
        axs[i, 3].yaxis.tick_right()
        axs[i, 3].xaxis.set_label_position('top')

        axs[i, 3].set(ylabel='Truth', xlabel='Prediction',
                      xticklabels=[f'≤ {thresholds[i]}', f'> {thresholds[i]}'],
                      yticklabels=[f'≤ {thresholds[i]}', f'> {thresholds[i]}'][::-1])

        axs[i, 3].text(0, 1.1, f'Test size: {len(x_test)}\n             ={test_size * 100:.0f}%\n',
                       transform=axs[i, 3].transAxes)
        axs[i, 3].text(0, -.6, f'Accuracy: {clf.score(x_test, y_test):.2f}',
                       transform=axs[i, 3].transAxes)

    wd = Path('.').resolve().parent / 'plots' / 'comp'
    Path.mkdir(wd, parents=True, exist_ok=True)
    fig.tight_layout()
    fig.savefig(wd / f'df__euclidean_{1+ 2*extend}_{modify}_{func.__name__}_{"-".join([str(t) for t in thresholds])}.png',
              dpi=300, bbox_inches=0)


In [12]:

extend = 5
modify = 'abs'
metric = 'euclidean'
thresholds = [3, 1, 1.5]

procs = []
i = 0
for modify in ['pos', 'flip', 'abs']:
    for func in [sum, max]:
        for thresholds in [[3,1,1.5], [10, 3, 3]]:
            proc = Process(target=do_work,
                           args=(extend, modify, metric, func, thresholds))
            proc.start()
            print(f'starting {i}')
            procs.append(proc)
            i += 1
            sleep(.5)
for proc in procs:
    proc.join()

reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 0
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 1
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 2
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 3
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 4
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 5
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 6
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 7
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 8
starting 9
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 10
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
starting 11
reading /home/quirin/PYTHON/mapra/all_sequences_prothermdb_HALF.h5 ...
read 8113 embe